In [1]:
import networkx as nx # version 2.2
import matplotlib.pyplot as plt
import re
import cvxpy as cp
import operator #to sort elements in a list of tuples
import itertools
import math
import numpy as np
import os
import sys
import time

import Init_NetRate as Init
import Cascade_generation_functions_NetRate as Gen

In [2]:
network_file_name = "./../NetInf_mycode/Generation_files/Gen_1_SG/G_1.txt"
cascade_file_name = "./../NetInf_mycode/Generation_files/Gen_1_SG/C_1.txt"
window = 10
beta = 0.5
model = 0
eps = 1e-7
rho = 0.95
gamma = 0.001 #step size


In [3]:
def Compute_grad_ij(alpha_mat,DAG,index):
    j,i = index
    t_i = DAG.nodes[i]["time"]
    t_j = DAG.nodes[j]["time"]
    parents = list(DAG.predecessors(i))
    sum_grad = 0
    for papa in parents :
        sum_grad += alpha_mat[papa,i]
    grad_ij = t_j-t_i - 1/sum_grad
    return grad_ij

In [34]:
G_true = Gen.Generate_random_graph(400,800)
Cascades = Gen.Generate_all_cascades(G_true,-2000,window,model,beta)
Gen.Save_cascade_to_file("Cascade_test_SG.txt",Cascades,G_true)
Gen.Save_graph_to_file("Graph_test_SG.txt",G_true)
G_star,DAG_C_dic = Init.Init("Cascade_test_SG.txt")
# G_true = Init.Load_ground_truth(network_file_name)
# G_star, DAG_C_dic = Init.Init(cascade_file_name)

All nodes were read


In [35]:
nb_c_per_nodes = np.zeros(G_true.number_of_nodes())
for c in DAG_C_dic :
    DAG = DAG_C_dic[c]
    for node in DAG :
        nb_c_per_nodes[node] +=1

In [52]:
# A_hat = np.random.uniform(0.01,1,(G_true.number_of_nodes(),G_true.number_of_nodes()))
A_hat = np.full((G_true.number_of_nodes(),G_true.number_of_nodes()),0.01)
for index,nb in enumerate(nb_c_per_nodes):
    if nb==0:
        A_hat[:,index]=eps
np.fill_diagonal(A_hat,eps)

In [53]:
k=0
while k <=1000 :
    c_index = np.random.randint(0,len(DAG_C_dic),1)[0]
    DAG_c = DAG_C_dic[c_index]
    if k%100==0:
        print(k)
    for j in G_true.nodes():
        for i in G_true.nodes():
#             if j not in DAG_c.nodes() and A_hat[j,i]>eps :
#                 A_hat[j,i] = max(rho *A_hat[j,i],eps)
            if j in DAG_c.nodes() and i not in DAG_c.nodes():
                t_j = DAG_c.nodes[j]["time"]
                A_hat[j,i] = max((A_hat[j,i]-gamma*(window-t_j)),eps)
            elif (j,i) in DAG_c.edges():
                grad = Compute_grad_ij(A_hat,DAG_c,(j,i))
                A_hat[j,i] = max((A_hat[j,i]-gamma*grad),eps)
    k+=1

0
100
200
300
400
500
600
700
800
900
1000


In [55]:
'''
Precision and recall computation
'''
correct = 0
edge_G_star = list(G_star.edges())
G_star.remove_edges_from(edge_G_star)
for i in range(0,G_true.number_of_nodes()):
    for j in range(0,G_true.number_of_nodes()):
        if A_hat[i,j] >1e-2:
            G_star.add_edge(i,j)
            if (i,j) in G_true.edges():
                correct +=1
#                 print("edge ",(i,j))
#                 print("alpha is ",A_hat[i,j])
print("Precision :",correct/G_star.number_of_edges())
print("Recall :",correct/G_true.number_of_edges())
print("Number of correct infered edges : ",correct)
print("Number of edges in G_hat :",G_star.number_of_edges())
print("Number of edges in the true network ",G_true.number_of_edges())

Precision : 0.19661016949152543
Recall : 0.87
Number of correct infered edges :  696
Number of edges in G_hat : 3540
Number of edges in the true network  800
